In [13]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pmdarima.arima import auto_arima
from datetime import datetime
import matplotlib.pyplot as plt
import tsfresh
from pmdarima.arima import ADFTest
from tsfresh import extract_features
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [14]:
values = read_csv("../../Time_series_data.csv")

In [15]:
values.head(13)

,Month,Sales
0,2013-01,2815
1,2013-02,2672
2,2013-03,2755
3,2013-04,2721
4,2013-05,2946
5,2013-06,3036
6,2013-07,2282
7,2013-08,2212
8,2013-09,2922
9,2013-10,4301


In [16]:

values = values.rename(columns={values.columns[1]: 'Data'})
values = values.rename(columns={values.columns[0]: 'Time'})
values['Time'] = pd.to_datetime(
    values['Time'], errors='coerce')

In [17]:
freq = pd.infer_freq(values["Time"])

In [18]:
# end_date = pd.to_datetime(x) + pd.DateOffset(months=10)
# print(end_date)

In [19]:
values.set_index('Time', inplace=True)

In [20]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(
    values, test_size=0.2, shuffle=False)

In [32]:
train = train[:80]
train.shape

(80, 1)

In [21]:
arima_model = auto_arima(train, start_p=0, d=1, start_q=0,
                            max_p=2, max_d=2, max_q=2, start_P=0,
                            D=1, start_Q=0, max_P=2, max_D=2,
                            max_Q=2, m=12, seasonal=True,
                            error_action='warn', trace=True,
                            supress_warnings=True, stepwise=True,
                            random_state=20, n_fits=50)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=1183.693, Time=0.02 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=1173.736, Time=0.17 sec


 ARIMA(0,1,1)(0,1,1)[12]             : AIC=1157.042, Time=0.32 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=1155.109, Time=0.06 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=1157.009, Time=0.22 sec
 ARIMA(0,1,1)(1,1,1)[12]             : AIC=1158.348, Time=0.53 sec
 ARIMA(1,1,1)(0,1,0)[12]             : AIC=1155.379, Time=0.14 sec
 ARIMA(0,1,2)(0,1,0)[12]             : AIC=1155.138, Time=0.18 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=1173.612, Time=0.05 sec
 ARIMA(1,1,2)(0,1,0)[12]             : AIC=1155.790, Time=0.29 sec
 ARIMA(0,1,1)(0,1,0)[12] intercept   : AIC=inf, Time=0.10 sec

Best model:  ARIMA(0,1,1)(0,1,0)[12]          
Total fit time: 2.088 seconds


In [24]:
test

,Data
Time,
2020-01-01,3934
2020-02-01,3162
2020-03-01,4286
2020-04-01,4676
2020-05-01,5010
2020-06-01,4874
2020-07-01,4633
2020-08-01,1659
2020-09-01,5951


In [26]:
test1 = test[:12]
test2 = test[12:]
test2

,Data
Time,
2021-01-01,4348
2021-02-01,3564
2021-03-01,4577
2021-04-01,4788
2021-05-01,4618
2021-06-01,5312
2021-07-01,4298
2021-08-01,1413
2021-09-01,5877


In [27]:
arima_model.fit(test1)

ARIMA(order=(0, 1, 1), scoring_args={}, seasonal_order=(0, 1, 0, 12),
      suppress_warnings=True, with_intercept=False)

In [28]:
n_periods = test2.shape[0]

In [29]:
prediction = pd.DataFrame(
    arima_model.predict(n_periods=n_periods), index=test2.index)
prediction.columns = ['predicted_values']

In [30]:
prediction

,predicted_values
Time,
2021-01-01,14637.000983
2021-02-01,13865.000983
2021-03-01,14989.000983
2021-04-01,15379.000983
2021-05-01,15713.000983
2021-06-01,15577.000983
2021-07-01,15336.000983
2021-08-01,12362.000983
2021-09-01,16654.000983


In [ ]:
n_periods = test.shape[0]

In [23]:
x = values.index[train.shape[0]]
print(x)

2020-01-01 00:00:00


In [ ]:
n_periods = test.shape[0]

In [ ]:
index_future_dates = pd.date_range(
    start=x, periods=n_periods, freq=freq)

index_future_dates

In [ ]:
predicted = model_fit.forecast(n_periods)

In [ ]:
prediction = pd.DataFrame(predicted, index=index_future_dates)
prediction.columns = ['predicted_values']
prediction.reset_index(inplace=True)

In [ ]:
# prediction=prediction.tail(12)
print(prediction)

In [ ]:
prediction.columns.value_counts

In [ ]:
plt.plot(values.index,values['Data'])
plt.plot(index_future_dates,predicted)

In [ ]:
mae = mean_absolute_error(test['Data'], prediction['predicted_values'])
mae

In [ ]:
mape = mean_absolute_percentage_error(test['Data'], prediction['predicted_values'])
mape

In [ ]:
mse = mean_squared_error(test['Data'], prediction['predicted_values'])
mse

In [ ]:
import itertools